In [28]:
# Google CLoud Storage
# !pip install google-cloud-storage

# Install geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [2]:
import ee, geemap
import math
import gc
import numpy as np
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4ZXFhzpcfP3dr5dE_PWAoto0iNmrNDSgcasjpWLMrLQ&tc=sXcDlAGpeUd6CSywyxjCNuevWVem-BlQmiHIX2zKELA&cc=IhkZaTMBZy3jv5RrvwKjHC19bv8hFqp-Sl1sMnrAJQo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qhl8r9XGT2qW8VKkNOXKyR6fcHUcogNB9fcKLhhfTrnv84STbb-LgA

Successfully saved authorization token.


In [3]:
def get_main_args():
    parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
    arg = parser.add_argument
    
    arg("--afghanistan_dir", type=str, default="users/Plottings/Afghanistan", help="Afghanistan Boundary Map Directory")
    arg("--burkina_faso_dir", type=str, default="users/Plottings/Burkina_Faso", help="Burkina Faso Boundary Map Directory")
    arg("--ethiopia_dir", type=str, default="users/Plottings/Ethiopia", help="Ethiopia Boundary Map Directory")
    arg("--ghana_dir", type=str, default="users/Plottings/Ghana", help="Ghana Boundary Map Directory")
    arg("--kenya_dir", type=str, default="users/Plottings/Kenya", help="Kenya Boundary Map Directory")
    arg("--senegal_dir", type=str, default="users/Plottings/Senegal", help="Senegal Boundary Map Directory")
    arg("--zambia_dir", type=str, default="users/Plottings/Zambia", help="Zambia Boundary Map Directory")    
    arg("--terra_LST_dir", type=str, default="MODIS/061/MOD11A1", help="Terra LST Image Collection")
    arg("--aqua_LST_dir", type=str, default="MODIS/061/MYD11A1", help="Aqua LST Image Collection")
    arg("--terra_NDVI_dir", type=str, default="MODIS/061/MOD13Q1", help="Terra NDVI Image Collection")
    arg("--aqua_NDVI_dir", type=str, default="MODIS/061/MYD13Q1", help="Aqua NDVI Image Collection")
    arg("--modis_ET_dir", type=str, default="MODIS/006/MOD16A2", help="ET Image Collection")
    arg("--precip_dir", type=str, default="NASA/GPM_L3/IMERG_V06", help="Precip Image Collection")
    arg("--sm_dir", type=str, default="NASA/GLDAS/V021/NOAH/G025/T3H", help="SM Image Collection")
    arg("--years", type=list, default=[2016+i for i in range(7)], help="Study Period Range-years")
    arg("--months", type=list, default=[1, 2, 3, 4], help="Study Period Range-months")
    arg("--season", type=str, default='Growing', help="Study Period Season")
    arg("--visualize", type=bool, default=True, help="Option to Display Maps")
    arg("--idx", type=int, default=20, help="Index of Image to Display")
    arg("--vciVis", type=dict, default={'min':0.0, 'max': 1.0,
                                        'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', 
                                        '99B718', '74A901','66A000', '529400', '3E8601', 
                                        '207401', '056201', '004C00', '023B01','012E01', 
                                        '011D01', '011301']}, help="Visualization Parameters of VCI")
    arg("--tciVis", type=dict, default={'min':0.0, 'max': 1.0,
                                        'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                                                    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                                                    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                                                    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                                                    'ff0000', 'de0101', 'c21301', 'a71001', '911003']}, help="Visualization Parameters of TCI")
    arg("--pciVis", type=dict, default={'min':0.0, 'max': 1.0,
                                        'palette': ['d7191c','fdae61','ffffbf','abdda4','2b83ba']}, help="Visualization Parameters of PCI")
    arg("--etciVis", type=dict, default={'min':0.0, 'max': 1.0,
                                        'palette': ['blue', 'orange', 'red']}, help="Visualization Parameters of ETCI")
    arg("--smciVis", type=dict, default={'min':0.0, 'max': 1.0,
                                        'palette': ['0300ff', '418504','efff07', 'efff07', 'ff0303']}, help="Visualization Parameters of SMCI") 
    arg("--cdmiVis", type=dict, default = {'min':0.0, 'max': 1.0,
                                           'palette':  ['#a50026','#d73027','#f46d43','#fdae61','#fee08b',
                                                        '#ffffbf','#d9ef8b','#a6d96a','#66bd63','#1a9850','#006837']}, help="Visualization Parameters of CMDI")
    arg("--scale", type=int, default=250, help="Downsampling Scale")
    arg("--bandNames", type=list, default=['VCI', 'TCI', 'PCI', 'ETCI', 'SMCI'], help="Downsampling Scale")


    return parser.parse_args([])


In [4]:
class GetIndices():
  def __init__(self, args, roi, index, sum=False):
    
    self.args = args
    self.sum = sum
    self.index = index
    self.roi = roi

    self.raw_data = self.filter_data()
    self.seasonal_data = self.seasonal_filter()
    self.monthly_agg  = self.monthly_Data()
    self.monthly_min, self.monthly_max = self.monthly_Min_Max()

  def filter_data(self):
    if self.index == 'TCI':
      terra_lst = ee.ImageCollection(self.args.terra_LST_dir)
      aqua_lst = ee.ImageCollection(self.args.terra_LST_dir)
      MODIS_LST = terra_lst.merge(aqua_lst)\
                           .filterBounds(self.roi)\
                           .select('LST_Day_1km') 
      return MODIS_LST.map(lambda img: img.multiply(0.02)\
                                          .subtract(273.15)\
                                          .float()\
                                          .set("system:time_start", img.get("system:time_start")))


    elif self.index == 'VCI': 
      terra_ndvi = ee.ImageCollection(self.args.terra_NDVI_dir)
      aqua_ndvi = ee.ImageCollection(self.args.terra_NDVI_dir)
      MODIS_NDVI = terra_ndvi.merge(aqua_ndvi)\
                             .filterBounds(self.roi) \
                             .select('NDVI')  
      return MODIS_NDVI.map(lambda img: img.divide(10000)\
                                            .float()\
                                            .set("system:time_start", img.get("system:time_start")))
                  
      
    elif self.index == 'ETCI':
      modis_et = ee.ImageCollection(self.args.modis_ET_dir)
      MODIS_ET = modis_et.filterBounds(self.roi) \
                         .select('ET')
      return MODIS_ET.map(lambda img: img.multiply(0.1)\
                                         .float()\
                                         .set("system:time_start", img.get("system:time_start")))                      


    elif self.index == 'PCI':
      precip = ee.ImageCollection(self.args.precip_dir)
      return precip.filterBounds(self.roi)\
                   .select('precipitationCal')            
    
    elif self.index == 'SMCI':
      sm = ee.ImageCollection(self.args.sm_dir)
      return sm.filterBounds(self.roi) \
               .select('SoilMoi10_40cm_inst') 

  
  def seasonal_filter (self): 
    if self.args.season == 'sowing':
      return self.raw_data.filter(ee.Filter.calendarRange(2016, 2021, 'year')) \
                          .filter(ee.Filter.calendarRange(11, 12, 'month'))
    else:
      return self.raw_data.filter(ee.Filter.calendarRange(2016, 2022, 'year')) \
                          .filter(ee.Filter.calendarRange(1, 4, 'month'))
  
  ## Map over the years and create a monthly aggregates (sum or mean) 
  def monthly_Data (self):
    monthly_data = []
    for year in self.args.years:
      for month in self.args.months:
        Monthly_data = self.seasonal_data.filter(ee.Filter.calendarRange(year, year, 'year')) \
                                         .filter(ee.Filter.calendarRange(month, month, 'month')) \

        if self.sum == False:
          monthly_data.append (Monthly_data.mean() \
                                           .set({'month': month, 'year': year}))
        else:
          monthly_data.append (Monthly_data.sum() \
                                           .set({'month': month, 'year': year}))      
    return ee.ImageCollection.fromImages(monthly_data)


  ## compute Min and Max for each month across all years
  def monthly_Min_Max(self):
    min, max = [], []
    for month in self.args.months:
      Monthly_min = self.monthly_agg.filter(ee.Filter.eq('month', month)) \
                                    .min()\
                                    .set('month', month)

      Monthly_max = self.monthly_agg.filter(ee.Filter.eq('month', month)) \
                                    .max()\
                                    .set('month', month)
      min.append(Monthly_min)
      max.append(Monthly_max)
    return ee.ImageCollection.fromImages(min), ee.ImageCollection.fromImages(max)

  def Compute_Index (self, image):
    # TCI = (max - avg) / (max - min)
    # VCI, PCI, ETCI = (avg - min) / (max - min)
    if self.index =='TCI':
      expression = '(max - avg) / (max - min)'
    else: 
      expression = '(avg - min) / (max - min)'
    
    return image.expression(expression,
                            {'avg': image.select('avg'),
                            'min': image.select('min'),
                            'max': image.select('max')
                            }).rename(self.index) 
  
  def get_scaled_index(self):
    Index_img = []
    for year in self.args.years:
      for month in self.args.months:
        filtered =  self.monthly_agg.filter(ee.Filter.eq('year', year)) \
                                    .filter(ee.Filter.eq('month', month))
        avg = ee.Image(filtered.first())
        min = ee.Image(self.monthly_min.filter(ee.Filter.eq('month', month)).first())
        max = ee.Image(self.monthly_max.filter(ee.Filter.eq('month', month)).first())

        image = ee.Image.cat([avg, min, max]) \
                        .rename(['avg', 'min', 'max'])
        
        ScaledIndex = self.Compute_Index (image).set('month', month).set('year', year)
        Index_img.append (ScaledIndex)

    return ee.ImageCollection.fromImages(Index_img)

In [5]:
# This helper function returns a list of new band names.
def getNewBandNames(prefix, bandNames):
  seq = ee.List.sequence(1, len(bandNames))
  return seq.map(lambda b: ee.String(prefix).cat(ee.Number(b).int().format()))


# This function accepts mean centered imagery, a scale and
# a region in which to perform the analysis.  It returns the
# Principal Components (PC) in the region as a new image.
def getPrincipalComponents(centered, scale, region, bandNames):
  # Collapse the bands of the image into a 1D array per pixel.
  arrays = centered.toArray()

  # Compute the covariance of the bands within the region.
  covar= arrays.reduceRegion(**{
    'reducer': ee.Reducer.centeredCovariance(),
    'geometry': region,
    'scale': scale,
    'maxPixels': 1e9
  })

  # Get the 'array' covariance result and cast to an array.
  # This represents the band-to-band covariance within the region.
  covarArray = ee.Array(covar.get('array'))

  # Perform an eigen analysis and slice apart the values and vectors.
  eigens = covarArray.eigen()

  # This is a P-length vector of Eigenvalues.
  eigenValues = eigens.slice(1, 0, 1)
  # This is a PxP matrix with eigenvectors in rows.
  eigenVectors = eigens.slice(1, 1)

  # Convert the array image to 2D arrays for matrix computations.
  arrayImage = arrays.toArray(1)

  # Left multiply the image array by the matrix of eigenvectors.
  principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

  # Turn the square roots of the Eigenvalues into a P-band image.
  sdImage = ee.Image(eigenValues.sqrt()) \
    .arrayProject([0]).arrayFlatten([getNewBandNames('sd', bandNames)])

  # Turn the PCs into a P-band image, normalized by SD.
  return principalComponents \
    .arrayProject([0]) \
    .arrayFlatten([getNewBandNames('pc', bandNames)]) \
    .divide(sdImage), eigenVectors

In [6]:
def compute_CMDI(VCI, TCI, PCI, ETCI, SMCI, weights, roi):
    wVCI = VCI.clip(roi).multiply(weights[0])
    wTCI = TCI.clip(roi).multiply(weights[1]) 
    wPCI = PCI.clip(roi).multiply(weights[2])
    wETCI = ETCI.clip(roi).multiply(weights[3]) 
    wSMCI = SMCI.clip(roi).multiply(weights[4])  
    
    return  wVCI.add(wTCI).add(wPCI).add(wETCI).add(wSMCI).rename('CMDI') 

In [7]:
args = get_main_args()

## **Define ROI**

In [ ]:
# countries: Afghanistan, Burkina Faso, Ethiopia, Ghana, Kenya, Senegal, Zambia
roi = ee.FeatureCollection(args.zambia_dir) 
Map = geemap.Map(plugin_Draw=True, Draw_export=False)
Map.centerObject(roi, 6)
Map.addLayer(roi, {}, ' Zambia Boundary Map')
Map

## **Getting data for the growing season**

In [33]:
TCI = GetIndices(args, roi, index='TCI', sum=False).get_scaled_index()
VCI = GetIndices(args, roi, index='VCI', sum=False).get_scaled_index()
ETCI = GetIndices(args, roi, index='ETCI', sum=True).get_scaled_index()
PCI  = GetIndices(args, roi, index='PCI', sum=True).get_scaled_index()
SMCI = GetIndices(args, roi, index='SMCI', sum=False).get_scaled_index()

listOfVCIImages = VCI.toList(VCI.size())
listOfTCIImages = TCI.toList(TCI.size())
listOfPCIImages = PCI.toList(PCI.size())
listOfETCIImages = ETCI.toList(ETCI.size())
listOfSMCIImages = SMCI.toList(SMCI.size())

args.idx = 10

VCI_image = ee.Image(listOfVCIImages.get(args.idx))
TCI_image = ee.Image(listOfTCIImages.get(args.idx))
PCI_image = ee.Image(listOfPCIImages.get(args.idx))
ETCI_image = ee.Image(listOfETCIImages.get(args.idx))
SMCI_image = ee.Image(listOfSMCIImages.get(args.idx))

In [34]:
if args.season == 'Growing':
  month = ['January', 'February', 'March', 'April']
  year = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
else:
  month = ['November', 'December']
  year = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
        
dates = [i+' '+j for j in year for i in month]
date = dates[args.idx]

In [28]:
Map = geemap.Map(zoom = 6, plugin_Draw=True, Draw_export=False)
Map.centerObject(roi, 6)
Map.addLayer(VCI_image.clip(roi), args.vciVis, 'VCI, '+date) 
Map.addLayer(TCI_image.clip(roi), args.tciVis, 'TCI, '+date)
Map.addLayer(PCI_image.clip(roi), args.pciVis, 'PCI, '+date) 
Map.addLayer(ETCI_image.clip(roi), args.etciVis, 'ETCI,'+date)

Map.addLayer(SMCI_image.clip(roi), args.smciVis, 'SMCI, '+date) 
Map.add_colorbar(args.vciVis, label="VCI", orientation="vertical", layer_name="VCI, "+date)
Map

Map(center=[-13.453394449965073, 27.828177406658938], controls=(WidgetControl(options=['position', 'transparen…

## **Compute the contribution coefficients using PCA**

In [35]:
gc.collect()

image = ee.Image.cat([VCI_image.clip(roi), 
                      TCI_image.clip(roi),
                      PCI_image.clip(roi),
                      ETCI_image.clip(roi),
                      SMCI_image.clip(roi)]) 

# Get the PCs at the specified scale and in the specified region
pcImage, eigenVectors = getPrincipalComponents(image, args.scale, roi, args.bandNames)    
eigenVectors_np = np.array(eigenVectors.getInfo())[0]
contrib_coeff = eigenVectors_np**2
weights = [math.ceil(i*100)/100 for i in contrib_coeff]
weights

[0.2, 0.14, 0.14, 0.15, 0.4]

## **Compute CMDI**

In [13]:
CMDI_image = compute_CMDI(VCI_image, TCI_image, PCI_image, ETCI_image, SMCI_image, weights, roi)

In [ ]:
Map = geemap.Map(zoom = 6, plugin_Draw=True, Draw_export=False)
Map.centerObject(roi, 6)
Map.addLayer(CMDI_image.clip(roi), args.cdmiVis, 'CMDI,'+date) 
Map.add_colorbar(args.cdmiVis, label="CMDI", orientation="vertical", layer_name="CMDI, "+date)
Map

In [25]:
img = SMCI_image.reduceRegion(reducer=ee.Reducer.toList(),\
                                        geometry=roi,\
                                        maxPixels=1e13,\
                                        scale=1000);

In [27]:
arr = np.array((ee.Array(img.get('SMCI')).getInfo())) 
gc.collect()
arr.max(), arr.min() 

(1.0, 0.0)